---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [91]:
# Import libraries
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

# train = pd.read_csv('readonly/train.csv')
# test = pd.read_csv('readonly/test.csv')
# address = pd.read_csv('readonly/addresses.csv')

In [92]:
#! cat readonly/test.csv > test.csv

In [93]:
test =pd.read_csv('test.csv', encoding = 'ISO-8859-1', index_col = 'ticket_id')

In [94]:
train = pd.read_csv('train.csv', encoding = 'ISO-8859-1', index_col = 'ticket_id')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [96]:
# address = pd.read_csv('readonly/addresses.csv', encoding = 'ISO-8859-1',  index_col = 'ticket_id')

In [97]:
# latlon = pd.read_csv('readonly/latlons.csv', encoding = 'ISO-8859-1')

In [98]:
train.dropna(subset = ['compliance'], inplace = True)

In [99]:
train['compliance'] = train['compliance'].astype(int)

In [100]:
common_col_to_drop = ['agency_name', 'inspector_name', 'violator_name', 'violation_street_number',
                      'violation_street_name', 'violation_zip_code', 'mailing_address_str_number', 'mailing_address_str_name', 
                      'city', 'state', 'zip_code', 'disposition', 'country', 'non_us_str_code', 'violation_description', 'grafitti_status', 'clean_up_cost']

In [101]:
train_col_to_drop = ['payment_status', 'payment_date', 'balance_due', 'payment_amount',
                    'collection_status', 'compliance_detail'] + common_col_to_drop

In [102]:
train = train.drop(train_col_to_drop, axis = 1)

### Violation Code
Count the frequency of unique violation code
<br>
Convert violation code to Categorical data type
<br>
Label Encoder

In [105]:
for df in [train, test]:
# Convert violation_code to categorical
    df['violation_code'] = pd.Categorical(df['violation_code'])
    violation_count = df['violation_code'].value_counts(normalize = True)
    ten_percent = violation_count[violation_count >= 0.1]
    violation_code = ten_percent.index.tolist()
    # Convert violation code less than 10% into other category
    df['violation_code'] = df['violation_code'].map(lambda x : 'Other' if x not in violation_code else x)
    labelencoder = LabelEncoder()
    df['violation_code'] = labelencoder.fit_transform(df['violation_code'])

### Ticket Date
Convert ticket_issued_data, hearing_date in to Datetime data type
<br>
Calculate the time difference
<br>
Categorize into within one month, late
<br>
Labelencode

In [106]:
for df in [train, test]:
    df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])
    df['hearing_date'] = pd.to_datetime(df['hearing_date'])
    df['Time Diff'] = df['hearing_date'] - df['ticket_issued_date']
    df['Time Diff'] = df['Time Diff'].dt.days
    df.drop(['ticket_issued_date', 'hearing_date'], inplace = True, axis = 1)
    df['Time Diff'] = df['Time Diff'].fillna(df['Time Diff'].mean())

### Gradient Boosting Classifier

In [108]:
test = test.drop(common_col_to_drop, axis = 1)

In [110]:
test.isnull().sum()

violation_code     0
fine_amount        0
admin_fee          0
state_fee          0
late_fee           0
discount_amount    0
judgment_amount    0
Time Diff          0
dtype: int64

In [112]:
X = train.loc[:, train.columns != 'compliance']
y = train['compliance']

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y , random_state = 0)

In [119]:
grid_values = {'learning_rate' : [0.01, 0.1, 1], 'max_depth' : [3, 4, 5]}
clf = GradientBoostingClassifier(random_state =0)
grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(X_train, y_train)
prob = grid_clf_auc.predict_proba(test)[:, 1]

In [120]:
roc_auc_score(y_test, grid_clf_auc.predict_proba(X_test)[:, 1])

0.77905091222220246

### Random Forest Classifier

In [129]:
def blight_model():
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import GridSearchCV
    
    # Import data
    train = pd.read_csv('train.csv', encoding = 'ISO-8859-1', index_col = 'ticket_id')
    test = pd.read_csv('test.csv', encoding = 'ISO-8859-1', index_col = 'ticket_id')
    # Data wrangling
    common_col_to_drop = ['agency_name', 'inspector_name', 'violator_name', 'violation_street_number',
                      'violation_street_name', 'violation_zip_code', 'mailing_address_str_number', 'mailing_address_str_name', 
                      'city', 'state', 'zip_code', 'disposition', 'country', 'non_us_str_code', 'violation_description', 'grafitti_status', 'clean_up_cost']
    train_col_to_drop = ['payment_status', 'payment_date', 'balance_due', 'payment_amount',
                    'collection_status', 'compliance_detail'] + common_col_to_drop
    train = train.drop(train_col_to_drop, axis = 1)
    train.dropna(subset = ['compliance'], inplace = True)
    for df in [train, test]:
# Convert violation_code to categorical
        df['violation_code'] = pd.Categorical(df['violation_code'])
        violation_count = df['violation_code'].value_counts(normalize = True)
        ten_percent = violation_count[violation_count >= 0.1]
        violation_code = ten_percent.index.tolist()
        # Convert violation code less than 10% into other category
        df['violation_code'] = df['violation_code'].map(lambda x : 'Other' if x not in violation_code else x)
        labelencoder = LabelEncoder()
        df['violation_code'] = labelencoder.fit_transform(df['violation_code'])
# Convert data time data
    for df in [train, test]:
        df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])
        df['hearing_date'] = pd.to_datetime(df['hearing_date'])
        df['Time Diff'] = df['hearing_date'] - df['ticket_issued_date']
        df['Time Diff'] = df['Time Diff'].dt.days
        df.drop(['ticket_issued_date', 'hearing_date'], inplace = True, axis = 1)
        df['Time Diff'] = df['Time Diff'].fillna(df['Time Diff'].mean())
# Test Data
    test = test.drop(common_col_to_drop, axis = 1)
# Model : GridSearchCV + Gradient Boosting Classifier
    X = train.loc[:, train.columns != 'compliance']
    y = train['compliance']
    grid_values = {'learning_rate' : [0.01, 0.1, 1], 'max_depth' : [3, 4, 5]}
    clf = GradientBoostingClassifier(random_state =0)
    grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
    grid_clf_auc.fit(X, y)
    prob = grid_clf_auc.predict_proba(test)[:, 1]
    result = pd.Series(prob, index = test.index)

    return result# Your answer here

In [130]:
blight_model()

KeyboardInterrupt: 